In [ ]:
from k3d import K3D
import urllib.request
import os.path
import vtk

filename= 'cow.vtp'

model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, 0.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)

if not os.path.isfile(filename):
    response = urllib.request.urlopen("https://raw.githubusercontent.com/naucoin/VTKData/master/Data/cow.vtp")
    f = open(filename, 'w')
    f.write(response.read().decode('utf8'))
    f.close()
    
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(filename)
reader.Update()

plot = K3D()
cow3d = K3D.vtkPolyData(reader.GetOutput(), model_matrix, color=0xff0000)
plot += cow3d
plot.display()

In [ ]:
plane=vtk.vtkPlane()
plane.SetOrigin(0,0,0)
plane.SetNormal(1,1,0)
 
cutter=vtk.vtkCutter()
cutter.SetCutFunction(plane)
cutter.SetInputConnection(reader.GetOutputPort())
cutter.Update()

FeatureEdges = vtk.vtkFeatureEdges()
FeatureEdges.SetInputConnection(cutter.GetOutputPort())
FeatureEdges.BoundaryEdgesOn()
FeatureEdges.FeatureEdgesOff()
FeatureEdges.NonManifoldEdgesOff()
FeatureEdges.ManifoldEdgesOff()
FeatureEdges.Update()

cutStrips = vtk.vtkStripper() ; #Forms loops (closed polylines) from cutter
cutStrips.SetInputConnection(cutter.GetOutputPort())
cutStrips.Update()

cutPoly = vtk.vtkPolyData() ; #This trick defines polygons as polyline loop
cutPoly.SetPoints((cutStrips.GetOutput()).GetPoints())
cutPoly.SetPolys((cutStrips.GetOutput()).GetLines())

In [ ]:
model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, -5.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)
plot += K3D.vtkPolyData(cutPoly, model_matrix, color=0x0000ff)